# Advanced Functional Concepts

This notebook dives deeper into the `returns` library, exploring powerful patterns for handling more complex functional workflows. These tools help keep your code flat, readable, and robust, even when dealing with multiple operations that can fail.

## 1. Composing Monads with `returns.do`

**The Problem:** When you have a sequence of operations that each return a `Result` or `Maybe`, you often need the output of one as the input for the next. This can lead to a deeply nested chain of `.bind()` calls, often called the "Pyramid of Doom," which is hard to read.

**The Solution:** The `returns.do` notation provides a way to write this sequential logic in a clean, flat, imperative-looking style, while still preserving the underlying functional and monadic guarantees. It's syntactic sugar for a series of `.bind()` calls.

In [ ]:
from returns.do import do
from returns.result import Failure, Result, Success


# Imagine three functions that can fail:
def step1() -> Result[int, str]:
    return Success(10)


def step2(n: int) -> Result[int, str]:
    if n > 5:
        return Success(n * 2)
    return Failure("Step 2 failed: input not > 5")


def step3(n: int) -> Result[str, str]:
    return Success(f"Final result is {n}")


# --- The 'Pyramid of Doom' with .bind() ---
pyramid_result = step1().bind(step2).bind(step3)
assert isinstance(pyramid_result, Success)
assert pyramid_result.unwrap() == "Final result is 20"
print("✅ `.bind()` chain works as expected.")


# --- The clean, flat style with `returns.do` ---
@do
def do_notation_workflow() -> Result[str, str]:
    # Each `<-` is like a `.bind()` call. If any step returns a Failure,
    # the entire workflow short-circuits and returns that Failure.
    val1 = yield from step1()
    val2 = yield from step2(val1)
    val3 = yield from step3(val2)
    return Success(val3)


do_result = do_notation_workflow()

# --- Verification ---
assert isinstance(do_result, Success)
assert do_result.unwrap() == "Final result is 20"
assert do_result == pyramid_result  # The results are identical
print("✅ `returns.do` notation produces the same result cleanly.")

## 2. Collapsing Nested Structures with `sequence`

**The Problem:** You have a list of `Result`s (or `Maybe`s), for example, from validating multiple inputs. You want to know if *all* of them were successful. The ideal outcome is to transform `List[Result[A, B]]` into `Result[List[A], B]`. This new structure would be `Success[List[A]]` if all individual results were `Success`, or the *first* `Failure[B]` encountered.

**The Solution:** `returns.iterables.sequence` does exactly this. It flips the types around.

In [ ]:
from returns.iterables import sequence
from returns.result import Result, Success

# --- Verification for Success case ---
all_success = [Success(1), Success(2), Success(3)]
sequenced_success = sequence(all_success)

assert isinstance(sequenced_success, Success)
assert sequenced_success.unwrap() == [1, 2, 3]
print("✅ `sequence` correctly handled a list of all Successes.")

# --- Verification for Failure case ---
one_failure = [
    Success(1),
    Failure("Error at item 2"),
    Success(3),  # This is ignored
    Failure("Error at item 4"),  # This is also ignored
]
sequenced_failure = sequence(one_failure)

assert isinstance(sequenced_failure, Failure)
assert sequenced_failure.failure() == "Error at item 2"
print("✅ `sequence` correctly short-circuited on the first Failure.")

## 3. Lifting Functions into a Monadic Context

**The Problem:** You have a regular, pure function that works on unwrapped values (e.g., `def add(a: int, b: int) -> int`). You also have your values wrapped in a monadic container like `Maybe` (e.g., `Maybe[int]`). How do you apply your pure function to the wrapped values without manually unwrapping them?

**The Solution:** We "lift" the pure function so it can operate on wrapped values. This is achieved through a combination of `.map()` (for the first argument) and `.apply()` (for subsequent arguments). `apply` is a special method that takes a wrapped value and applies it to a wrapped function.

In [ ]:
from returns.curry import curry
from returns.maybe import Nothing, Some


# A regular, pure function:
@curry
def add(a: int, b: int) -> int:
    return a + b


# Our wrapped values:
maybe_a = Some(10)
maybe_b = Some(5)
maybe_c = Nothing

# --- Lifting `add` to work on Maybe ---
# 1. Use `.map()` to apply the first argument. This returns a wrapped function: `Some(add(10))`
lifted_add = maybe_a.map(add)

# 2. Use `.apply()` to apply the second argument to the wrapped function.
result_success = lifted_add.apply(maybe_b)

# --- Verification for Success case ---
assert isinstance(result_success, Some)
assert result_success.unwrap() == 15
print(f"✅ Lifted function correctly produced {result_success}")

# --- Verification for Failure case (if one of the inputs is Nothing) ---
result_failure = lifted_add.apply(maybe_c)
assert isinstance(result_failure, Nothing)
print(
    f"✅ Lifted function correctly produced {result_failure} when one input was Nothing."
)